In [24]:
import pandas as pd
import numpy as np
import torch
import re

In [18]:
filename1 =r'./data/ml-1m/users.dat'
filename2 = r'./data/ml-1m/ratings.dat'
filename3 = r'./data/ml-1m/movies.dat'
uname = ['user_id','gender','age','occupation','zip']
df_users = pd.read_table(filename1, sep='::', header = None, names=uname, engine='python')
rnames = ['user_id','movie_id','rating','timestamp']
df_ratings = pd.read_table(filename2, header =None, sep='::',names=rnames, engine= 'python')
mnames = ['movie_id','title','genres']  # genres 表示影片的体裁是什么
df_movies = pd.read_table(filename3, header = None, sep='::', names = mnames, engine='python',encoding='ISO-8859-1')

In [25]:
def load_data():
    """
    Load Dataset from File
    """
    #读取User数据
    users_title = ['UserID', 'Gender', 'Age', 'JobID', 'Zip-code']
    users = pd.read_table('./data/ml-1m/users.dat', sep='::', header=None, names=users_title, engine = 'python')
    users = users.filter(regex='UserID|Gender|Age|JobID')
    users_orig = users.values
    #改变User数据中性别和年龄
    gender_map = {'F':0, 'M':1}
    users['Gender'] = users['Gender'].map(gender_map)

    age_map = {val:ii for ii,val in enumerate(set(users['Age']))}
    users['Age'] = users['Age'].map(age_map)

    #读取Movie数据集
    movies_title = ['MovieID', 'Title', 'Genres']
    movies = pd.read_table('./data/ml-1m/movies.dat', sep='::', header=None, names=movies_title, engine = 'python',encoding='ISO-8859-1')
    movies_orig = movies.values
    #将Title中的年份去掉
    pattern = re.compile(r'^(.*)\((\d+)\)$')

    title_map = {val:pattern.match(val).group(1) for ii,val in enumerate(set(movies['Title']))}
    movies['Title'] = movies['Title'].map(title_map)

    #电影类型转数字字典
    genres_set = set()
    for val in movies['Genres'].str.split('|'):
        genres_set.update(val)

    genres_set.add('<PAD>')
    genres2int = {val:ii for ii, val in enumerate(genres_set)}

    #将电影类型转成等长数字列表，长度是18
    genres_map = {val:[genres2int[row] for row in val.split('|')] for ii,val in enumerate(set(movies['Genres']))}

    for key in genres_map:
        for cnt in range(max(genres2int.values()) - len(genres_map[key])):
            genres_map[key].insert(len(genres_map[key]) + cnt,genres2int['<PAD>'])
    
    movies['Genres'] = movies['Genres'].map(genres_map)

    #电影Title转数字字典
    title_set = set()
    for val in movies['Title'].str.split():
        title_set.update(val)
    
    title_set.add('<PAD>')
    title2int = {val:ii for ii, val in enumerate(title_set)}

    #将电影Title转成等长数字列表，长度是15
    title_count = 15
    title_map = {val:[title2int[row] for row in val.split()] for ii,val in enumerate(set(movies['Title']))}
    
    for key in title_map:
        for cnt in range(title_count - len(title_map[key])):
            title_map[key].insert(len(title_map[key]) + cnt,title2int['<PAD>'])
    
    movies['Title'] = movies['Title'].map(title_map)

    #读取评分数据集
    ratings_title = ['UserID','MovieID', 'ratings', 'timestamps']
    ratings = pd.read_table('./data/ml-1m/ratings.dat', sep='::', header=None, names=ratings_title, engine = 'python',encoding='ISO-8859-1')
    ratings = ratings.filter(regex='UserID|MovieID|ratings')

    #合并三个表
    data = pd.merge(pd.merge(ratings, users), movies)
    
    #将数据分成X和y两张表
    target_fields = ['ratings']
    features_pd, targets_pd = data.drop(target_fields, axis=1), data[target_fields]
    
    features = features_pd.values
    targets_values = targets_pd.values
    
    return title_count, title_set, genres2int, features, targets_values, ratings, users, movies, data, movies_orig, users_orig

In [26]:
load_data()

(15,
 {'A',
  'Sent-Down',
  'Trek',
  'Mamma',
  'Hammer',
  'Aimée',
  'Brown)',
  'Whole',
  'sidste',
  'Arrest',
  'rennt)',
  'Maîtres',
  'Fled',
  'Roseanna)',
  'Education',
  'America',
  'Unknown',
  'Grandview,',
  'Confessional,',
  'anges)',
  'Legends',
  'Legend',
  'Curse',
  'Clue',
  'Woop-Woop',
  'Shield,',
  'Yesterday',
  'Rent-a-Kid',
  'Earth',
  'Phantasm',
  'Videotape',
  'Terms',
  'Stinkers',
  'Designated',
  'Della',
  'Civil',
  'dans',
  'Tinseltown',
  'Sex',
  'Detective,',
  'Can',
  'Cowboy',
  'Ciao,',
  'Object',
  'Insomnia',
  'Exotica',
  'Carnosaur',
  'Mark',
  'Masters',
  'Vagabond',
  'Proposal',
  'Scent',
  'Blew',
  'Angels',
  'Flintstones,',
  'Naturally',
  'Tickle',
  'Nobody',
  'Blows,',
  'Bible,',
  'sauvage)',
  '(Mononoke',
  'Kiss,',
  'Reaction',
  'After,',
  'Vie)',
  'Lost',
  '(Her',
  'Party',
  '(En',
  '(Gojira)',
  'Turbo:',
  'Cleaning',
  'Fievel',
  'Top',
  'Sorority',
  'Cat,',
  'Gorgeous',
  'Against',
  'U-5